In [261]:
import math
import numpy as np
from numpy import genfromtxt
import operator

In [262]:
# import X and y from csv file
def read_logistic_data(filename):
    my_data = genfromtxt(filename, delimiter=';')
    X = []
    y= []
    for i in range(len(my_data)):
        new_data = my_data[i]
        y.append([new_data[-1]])
        X.append(np.delete(new_data, -1))
    return X, y

X_train, y_train = read_logistic_data('digits123-1.csv')
X_test, y_test = read_logistic_data('digits123-2.csv')

In [290]:
'''
INPUT: 
- training data (features and class concatenated)

OUTPUT:
- dictionary where classes are keys, with training examples
  corresponding to that class as values.
'''
def separate_classes(data):
    classdict = dict()
    for i in range(len(data)):
        x = data[i]
        if (x[-1] not in classdict):
            classdict[x[-1]] = []
        classdict[x[-1]].append(x)
    
    return classdict

classes = separate_classes(np.concatenate((X_train, y_train), axis=1))

In [285]:
print len(classes[1.0])

102


In [264]:
'''
INPUT:
- numpy array

OUTPUT:
- mean of values in array
- variance of values in array
'''
def mean_var(x):
    mean = sum(x)/len(x) 
    variance = sum(((x-mean)**2))/len(x)   
    return mean, variance    

In [265]:
'''
INPUT:
- class

OUTPUT:
- dictionary where classes are keys, with (mean, variance) list
  of training examples corresponding to correct class.
'''
def mean_var_class(c):
    mean_var_list = list()
    
    # loop over every attribute
    for i in range(len(classes[c][0])-1):
        mean_var_attribute = np.array([])
        # loop over every training example
        for j in range(len(classes[c])):
            mean_var_attribute = np.concatenate((mean_var_attribute, np.array([classes[c][j][i]])), axis=0)
        mean_var_list.append(mean_var(mean_var_attribute))
    
    return mean_var_list


# create dict: key = class, entry = list with mean and variance for every feature    
mean_var_dict = {1.0 : mean_var_class(1.0), 2.0: mean_var_class(2.0), 3.0 : mean_var_class(3.0)}

In [266]:
'''
INPUT:
- mean
- variance
- value for feature x

OUTPUT:
- probability density function
'''
def pdf(mean, variance, x):
    # ignore training examples with value 0 for mean and variance,
    # i.e. return p = 1, so that multiplying with p does not change value.
    if ((mean == 0) & (variance == 0)):
        p = 1
    else:
        p = (1/(math.sqrt(2*variance*math.pi)))*math.e**-(((x-mean)**2)/(2*variance))
    
    return p

In [345]:

'''
INPUT:
- list with classes
- test instance x

OUTPUT:
- dictionary where classes are keys, with p that x
  is in class as value.
'''
def p_x_given_class(classes, x):
    probabilities = {}
    for c in classes:     
        p = 1
        for i in range(len(x)):
            feature_train = x[i]
            feature_mean = mean_var_dict[c][i][0]
            feature_var = mean_var_dict[c][i][1]
            p *= pdf(feature_mean, feature_var, feature_train)
        probabilities[c] = p
            
    return probabilities

'''
INPUT:
- list with classes
- test instance x

OUTPUT:
- prediction for class
'''
def prediction(classes, x):
    # return highest probability found in class_probabilities
    
    # get p 1.0
    # get p 2.0
    # get p 3.0
    
    p_x_given_c = p_x_given_class(classes,x)
    
    # find n of training examples
    length = 0
    for i in classes:
        length += len(classes[i])
        
    # find value for P(x|c1) + P(x|c2) + ... + P(x|cn)
    divide_by = 0    
    for i in classes:
        divide_by += p_x_given_c[i] * len(classes[i])/float(length)
         
    for i in p_x_given_c:
        p_x_given_c[i] = ((p_x_given_c[i] * len(classes[i])/float(length)) / divide_by)
        
        
    return max(p_x_given_c.iteritems(), key=operator.itemgetter(1))[0]

print class_probabilities(classes, X_train[0])
print prediction(classes, X_train[0])

{1.0: 3.952058804347397e-47, 2.0: 2.1858205950974448e-64, 3.0: 1.8196239624211918e-63}
{1.0: 1.0, 2.0: 5.2054967167235808e-18, 3.0: 4.6042431363104841e-17}
1.0


In [346]:
'''
INPUT:
- test data: features
- test data: correct classes

OUTPUT:
- n of correctly classified instances
- n of incorrectly classified instances
- list with index of incorrectly classified instances
'''
def evaluation_gaussian_nb(X_test, y_test):
    true = 0
    false = 0   
    false_instances = list()
    for i in range(len(X_test)):
        predicted = prediction(classes, X_test[i])
        actual = y_train[i][0]
        if predicted == actual:
            true += 1
        if predicted != actual:
            false += 1
            false_instances.append(i)
             
    return true, false, false_instances

true, false, false_instances = evaluation_gaussian_nb(X_test, y_test)

print 'Correct:   ', true
print 'False:     ', false
print 'Accurracy: ', round((float(true)/float(false + true)), 5)    

{1.0: 1.0, 2.0: 7.4041777680808904e-51, 3.0: 3.2601791794601989e-38}
{1.0: 1.0, 2.0: 8.3926161273257002e-60, 3.0: 1.2593499065116449e-58}
{1.0: 1.0, 2.0: 4.1128963982878158e-79, 3.0: 1.8804826555684186e-59}
{1.0: 1.0, 2.0: 1.4568128049107501e-55, 3.0: 2.0358454350861484e-67}
{1.0: 1.0, 2.0: 3.5929171957420102e-94, 3.0: 1.0027683022139726e-80}
{1.0: 1.0, 2.0: 7.3569638236332712e-74, 3.0: 7.0361117113317249e-75}
{1.0: 1.0, 2.0: 2.1628923166638732e-81, 3.0: 2.1119605759226127e-78}
{1.0: 1.0, 2.0: 2.030466195835715e-62, 3.0: 2.6534782399191792e-56}
{1.0: 1.0, 2.0: 1.0134755012237835e-66, 3.0: 2.2861216101455133e-73}
{1.0: 1.0, 2.0: 2.6733827358994517e-56, 3.0: 2.5120208008616203e-48}
{1.0: 1.0, 2.0: 5.5388539934741223e-84, 3.0: 1.5925650712890672e-82}
{1.0: 1.0, 2.0: 7.6716058742065369e-36, 3.0: 3.7469087926467767e-59}
{1.0: 1.0, 2.0: 2.5480506410940118e-56, 3.0: 7.7634442149294399e-63}
{1.0: 1.0, 2.0: 1.2319781477520207e-161, 3.0: 1.3337210550313167e-134}
{1.0: 1.0, 2.0: 4.719774450591534

In [344]:
# pretty print misclassified instances to obtain a better understanding of underlying issues.
c = classes
for i in false_instances:
    print class_probabilities(c, X_test[i])
    print 'prediction: ', prediction(c, X_test[i])
    print 'actual:     ',y_train[i][0]
    print '------------------------------------------------------------------------'
    
# add prior probabilities!!!!!!! ALGORITHM SEE SHARELATEX
    

 {1.0: 8.7727579919187879e-68, 2.0: 7.6757727494372779e-66, 3.0: 0.0}
prediction:  2.0
actual:      1.0
------------------------------------------------------------------------
{1.0: 3.001399040509384e-79, 2.0: 2.3129165503814674e-70, 3.0: 0.0}
prediction:  2.0
actual:      1.0
------------------------------------------------------------------------
{1.0: 3.2307747381093167e-70, 2.0: 1.6815889701900517e-59, 3.0: 4.3221550669138766e-96}
prediction:  2.0
actual:      1.0
------------------------------------------------------------------------
{1.0: 1.7349709895829556e-71, 2.0: 4.0829056283834716e-48, 3.0: 3.1084867736730191e-71}
prediction:  2.0
actual:      1.0
------------------------------------------------------------------------
{1.0: 1.0158273915718126e-97, 2.0: 3.1710937831898381e-111, 3.0: 1.5007752556678846e-86}
prediction:  3.0
actual:      1.0
------------------------------------------------------------------------
{1.0: 7.4253158850782516e-159, 2.0: 1.7929702473286409e-73, 3.